In [8]:
import pandas as pd

with open("./avg-test-result-in-paper", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, engine='python')
df.head()

,file_system,dup_rate,num_job,bandwidth(MiB/s)
0,Light-Dedup,0,1,751
1,Light-Dedup,0,2,1124
2,Light-Dedup,0,4,1496
3,Light-Dedup,0,8,1647
4,Light-Dedup,0,16,1442


- How much Light-Dedup outperform other deduplication file systems under 0, 25, 50, 75 duplication ratio?

In [9]:
fs_names = df['file_system'].drop_duplicates().reset_index().drop(columns=['index'])['file_system']

light_dedup =  df[df["file_system"] == "Light-Dedup"].copy().reset_index()

for fs_name in fs_names:
    if fs_name == "Light-Dedup":
        continue
    fs = df[df["file_system"] == fs_name]
    # print(pd.DataFrame([1, 2, 3]) / pd.DataFrame([1, 2, 3]))
    light_dedup[fs_name] = light_dedup["bandwidth(MiB/s)"] / fs["bandwidth(MiB/s)"].reset_index()["bandwidth(MiB/s)"]

light_dedup
    

,index,file_system,dup_rate,num_job,bandwidth(MiB/s),Light-Dedup(SHA256),NV-Dedup,NOVA
0,0,Light-Dedup,0,1,751,2.781481,0.888757,0.866205
1,1,Light-Dedup,0,2,1124,2.412017,0.857361,0.829520
2,2,Light-Dedup,0,4,1496,1.842365,0.807339,0.855835
3,3,Light-Dedup,0,8,1647,1.534949,1.004881,0.879808
4,4,Light-Dedup,0,16,1442,1.129209,1.099924,0.936972
5,20,Light-Dedup,25,1,741,2.714286,1.337545,0.812500
6,21,Light-Dedup,25,2,1199,2.412475,1.236082,0.892777
7,22,Light-Dedup,25,4,1774,1.962389,1.133546,1.016619
8,23,Light-Dedup,25,8,1969,1.417567,1.108671,1.049574
9,24,Light-Dedup,25,16,1554,0.975518,1.079917,1.003228


- How much throughput Light-Dedup achieves over NOVA and NV-Dedup when dedup ratio >= 25%?

In [10]:
print("Max higher than NV-Dedup:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Min higher than NV-Dedup:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NV-Dedup"])))
print("Max higher than NOVA:" + str(max(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))
print("Min higher than NOVA:" + str(min(light_dedup[light_dedup["dup_rate"] >= 25]["NOVA"])))

Max higher than NV-Dedup:2.119700748129676
Min higher than NV-Dedup:0.9713592233009709
Max higher than NOVA:2.149637442320369
Min higher than NOVA:0.8125


- How much slower Light-Dedup than NOVA when dedup ratio == 0%?

In [11]:
light_dedup[light_dedup["dup_rate"] == 0]["NOVA"]

0    0.866205
1    0.829520
2    0.855835
3    0.879808
4    0.936972
Name: NOVA, dtype: float64

- How much faster Light-Dedup than NV-Dedup when dedup ratio == 75%?

In [12]:
light_dedup[light_dedup["dup_rate"] == 75]["NV-Dedup"]

15    2.119701
16    1.975904
17    1.828632
18    1.616274
19    1.102434
Name: NV-Dedup, dtype: float64

- How much faster Light-Dedup than SHA256 when dedup ratio == 75%?

In [13]:
light_dedup[(light_dedup["dup_rate"] == 75) & (light_dedup["num_job"] == 1)]["Light-Dedup(SHA256)"]

15    2.335165
Name: Light-Dedup(SHA256), dtype: float64

- Performance Improvement of Light-Dedup from 0% to 75% 

In [14]:
light_dedup[(light_dedup["dup_rate"] == 75)].reset_index()["bandwidth(MiB/s)"]/light_dedup[(light_dedup["dup_rate"] == 0)].reset_index()["bandwidth(MiB/s)"]

0    1.131824
1    1.313167
2    1.733289
3    2.291439
4    2.261442
Name: bandwidth(MiB/s), dtype: float64